In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
import sentencepiece as spm

In [3]:
# 모델 하이퍼파라메터와 저장 위치 등 설정 정보 선언 
from ratsnlp.nlpbook.classification import ClassificationDeployArguments

args = ClassificationDeployArguments(
    pretrained_model_name="beomi/kcbert-base",
    downstream_model_dir="./checkpoint-doccls",
    max_seq_length=128,
)

c:\Users\BIG2-003-008\.conda\envs\ratsnlp\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


downstream_model_checkpoint_fpath: ./checkpoint-doccls\epoch=1-val_loss=0.27.ckpt


In [4]:
# 파인튜닝을 마친 모델과 토크나이저를 읽어들임 
import torch
from transformers import BertConfig, BertForSequenceClassification

fine_tuned_model_ckpt = torch.load(
    args.downstream_model_checkpoint_fpath,
    map_location=torch.device("cpu")
)
pretrained_model_config = BertConfig.from_pretrained(
    args.pretrained_model_name,
    num_labels=fine_tuned_model_ckpt['state_dict']['model.classifier.bias'].shape.numel(),
)
model = BertForSequenceClassification(pretrained_model_config)
model.load_state_dict({k.replace("model.", ""): v for k, v in fine_tuned_model_ckpt['state_dict'].items()})
model.eval()

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30000, 768, padding_idx=0)
      (position_embeddings): Embedding(300, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0-11): 12 x BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12,

In [5]:
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained(
    args.pretrained_model_name,
    do_lower_case=False,
)

In [6]:
# 인퍼런스 함수 선언 
def inference_fn(sentence):
    inputs = tokenizer(
        [sentence],
        max_length=args.max_seq_length,
        padding="max_length",
        truncation=True,
    )
    with torch.no_grad():
        outputs = model(**{k: torch.tensor(v) for k, v in inputs.items()})
        prob = outputs.logits.softmax(dim=1)
        positive_prob = round(prob[0][1].item(), 4)
        negative_prob = round(prob[0][0].item(), 4)
        pred = "긍정 (positive)" if torch.argmax(prob) == 1 else "부정 (negative)"
    return {
        'sentence': sentence,
        'prediction': pred,
        'positive_data': f"긍정 {positive_prob}",
        'negative_data': f"부정 {negative_prob}",
        'positive_width': f"{positive_prob * 100}%",
        'negative_width': f"{negative_prob * 100}%",
    }

In [20]:
inference_fn('선생님 강의를 듣고 한 달 공부하여 96점을 받았습니다.')

{'sentence': '선생님 강의를 듣고 한 달 공부하여 96점을 받았습니다.',
 'prediction': '긍정 (positive)',
 'positive_data': '긍정 0.6643',
 'negative_data': '부정 0.3357',
 'positive_width': '66.43%',
 'negative_width': '33.57%'}

In [17]:
inference_fn('아 진짜 짜증나')['prediction'].split(' ')[0]

'부정'

In [ ]:
dongjin = pd.read_csv('dongjin.csv')

In [27]:
dongjin['감정'] = ''
dongjin['감정정답'] = ''

In [28]:
dongjin['감정'] = dongjin['수강후기내용'].apply(lambda x: inference_fn(x)['prediction'].split(' ')[0])

In [29]:
dongjin

,수강후기내용,감정,감정정답
0,선생님께서 문제 풀이 포인트를 잘 알려주셔서 큰 어려움 없이 화작을 풀 수 있었습니다.,부정,
1,교재 분량도 적어서 부담이 적었고요.,부정,
2,모고 때 항상 한두 개씩 틀려서 불안했는데 수능 때 1개만 틀린 걸로 만족하려고요.,부정,
3,비문학에서 많이 틀린 게 아쉽네요.,부정,
4,언매 고3 기출을 풀때 반은 틀리고 반은 찍어서맞고 이런 상태였고 화작은 하고싶지가...,부정,
5,강의할 때 세세하게 설명을 해주셔서 정말 좋았어요,긍정,
6,요즘은 기출을 풀어도 집중안해서 제대로 안읽어서이건 제가 고쳐야하는 부분 틀리는 것...,부정,
7,이제 문학 연계작품은 틀릴이유가 없습니다.,긍정,
8,수능 잘보고 돌아올게여,긍정,
9,일단 방동진 쌤을 모르고 있었던 고1인데 제가 자퇴를 해서 재수학원에 들어와서 쌤 ...,긍정,


In [30]:
dongjin.to_csv('dongjin__.csv')

In [42]:
dongjins = pd.read_csv('dongjins(정확도).csv')

In [47]:
dongjins.drop(labels=['Unnamed: 0'],axis=1,inplace=True)

In [48]:
dongjins

,수강후기내용,감정,감정정답
0,선생님께서 문제 풀이 포인트를 잘 알려주셔서 큰 어려움 없이 화작을 풀 수 있었습니다.,부정,긍정
1,교재 분량도 적어서 부담이 적었고요.,부정,긍정
2,모고 때 항상 한두 개씩 틀려서 불안했는데 수능 때 1개만 틀린 걸로 만족하려고요.,부정,긍정
3,비문학에서 많이 틀린 게 아쉽네요.,부정,부정
4,강의할 때 세세하게 설명을 해주셔서 정말 좋았어요,긍정,긍정
5,요즘은 기출을 풀어도 집중안해서 제대로 안읽어서이건 제가 고쳐야하는 부분 틀리는 것...,부정,긍정
6,이제 문학 연계작품은 틀릴이유가 없습니다.,긍정,긍정
7,일단 방동진 쌤을 모르고 있었던 고1인데 제가 자퇴를 해서 재수학원에 들어와서 쌤 ...,긍정,긍정
8,감사하다는 말씀드리고 싶고 정말 하나하나 세심하게 알려주셔서 이해도 잘되고 재밌게 ...,긍정,긍정
9,국어를 좋아하는 편도 아니고 원래 문법이 약하고 어려웠었는데 저도 언매를 다 들었다...,긍정,긍정


In [50]:
sum(dongjins['감정'] == dongjins['감정정답'])

12

In [8]:
taeseong = pd.read_csv('taeseong.csv')
taeseong['감정'] = ''
taeseong['감정정답'] = ''
taeseong['감정'] = taeseong['수강후기내용'].apply(lambda x: inference_fn(x)['prediction'].split(' ')[0])

In [9]:
taeseong

,수강후기내용,감정,감정정답
0,최상입니다~한국사 시험 100점으로 1급 ~^^수능 보기 전 최고의 선택이었던 것 ...,긍정,
1,잘보고오겠습니다 화이팅!이해하기 쉽고 재밌게 잘 지도해주십니다!감사합니다 최태성 선생님.,긍정,
2,선생님 강의를 듣고 한 달 공부하여 96점을 받았습니다.,긍정,
3,시험 치고 바로 선생님의 책 역사의 쓸모도 읽었구요.,긍정,
4,ㅎㅎ 선생님께서 항상 말씀하신대로 한능검 심화 1급이라 당당하게 말하고 생각할 수 ...,긍정,
...,...,...,...
293,63회 한능검 준비하고 있는 학생입니다.,긍정,
294,다른 과목을 공부하고 언제나 끝은 이 강의로 끝마쳐요!,긍정,
295,항상 느끼는건데 그 어떤 강의보다 진실함이 있으신 것 같고 노력하는 모습이 보기 좋...,긍정,
296,!강의 듣기 전 보다 내신등급은 2등급 올랐고 모의고사는 항상 2등급 유지에요!!,부정,


In [10]:
taeseong.to_csv('taeseong__.csv')